In [3]:
pip install nltk

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import keras

In [6]:
news_final=pd.read_csv("DataSet/News_Final.csv")

news_final.describe()
print(news_final.shape)

(93239, 11)


In [7]:
news_final[["Facebook","GooglePlus","LinkedIn"]] = news_final[["Facebook","GooglePlus","LinkedIn"]] .replace(-1, np.NaN)
news_final.dropna(inplace=True)
print(news_final.shape)

(81406, 11)


In [8]:
#Data cleaning
#removing unnecessary punctuations and tags

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    
    return input_txt

In [9]:
news_final['Title'] = np.vectorize(remove_pattern)(news_final['Title'], "@[\\w]*")

#replacing numbers and other special characters with a space in the Title elements
news_final['Title'] = news_final['Title'].str.replace("[^a-zA-Z#]", " ")

news_final['Headline'] = np.vectorize(remove_pattern)(news_final['Headline'], "@[\\w]*")
news_final['Headline'] = news_final['Headline'].str.replace("[^a-zA-Z#]", " ")

#remove stopwords
stop = stopwords.words('english')
news_final= pd.DataFrame(news_final)
news_final['Title'] = news_final['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


stop = stopwords.words('english')
news_final= pd.DataFrame(news_final)
news_final['Headline'] = news_final['Headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


C:\Users\Krish\AppData\Local\Temp/ipykernel_20428/63435972.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  news_final['Title'] = news_final['Title'].str.replace("[^a-zA-Z#]", " ")
C:\Users\Krish\AppData\Local\Temp/ipykernel_20428/63435972.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  news_final['Headline'] = news_final['Headline'].str.replace("[^a-zA-Z#]", " ")


In [10]:
news_final.dtypes

IDLink               float64
Title                 object
Headline              object
Source                object
Topic                 object
PublishDate           object
SentimentTitle       float64
SentimentHeadline    float64
Facebook             float64
GooglePlus           float64
LinkedIn             float64
dtype: object

In [11]:
z=[]
z=list(zip(news_final['IDLink'],news_final['Title'],news_final['Headline'],news_final['Source'],news_final['Topic'],news_final['Facebook']))
z1=sorted(z, key=lambda t: t[5])

labels = ['IDLink','Title', 'Headline','Source', 'Topic', 'SharesOfFacebook']
df_facebook = pd.DataFrame.from_records(z1, columns=labels)

def label_emotion (row):
   if row['SharesOfFacebook'] >=0 and row['SharesOfFacebook'] <=1050  :
      return 1
   if row['SharesOfFacebook'] > 1050 and row['SharesOfFacebook']<=15000:
      return 2
   if row['SharesOfFacebook'] > 15000 and row['SharesOfFacebook']<=20000:
      return 3
   if row['SharesOfFacebook'] >20000 and row['SharesOfFacebook'] <=50000:
      return 4
df_facebook['category_facebook'] = df_facebook.apply (lambda row: label_emotion (row),axis=1)

df_facebook

,IDLink,Title,Headline,Source,Topic,SharesOfFacebook,category_facebook
0,311.0,Microsoft offering discount original Band UK p...,Just ahead market rollout Microsoft new Band U...,International Business Times via Yahoo UK & Ir...,microsoft,0.0,1
1,252.0,Economy improve though change last months Asso...,quot In coming six months seems growing optimi...,IANS India Private Limited/Yahoo India News vi...,economy,0.0,1
2,227.0,Economy improve though change last months Asso...,New Delhi Nov IANS Majority Indian businesses ...,IANS via Yahoo Maktoob News,economy,0.0,1
3,307.0,Microsoft scales back free online storage plans,NEW YORK Microsoft getting stingy online stora...,The Columbian,microsoft,0.0,1
4,272.0,Pak economy firm level perspective Business Re...,The latest annual report Securities Exchange C...,Business Recorder,economy,0.0,1
...,...,...,...,...,...,...,...
81401,12561.0,For Wealthiest Private Tax System That Saves T...,A Winters political scientist Northwestern Uni...,New York Times,economy,29564.0,4
81402,72045.0,Obama legacy stake,Sreedhar Potarazu ophthalmologist entrepreneur...,CNN,obama,30489.0,4
81403,76901.0,I Miss Barack Obama,As primary season gone along strange sensation...,New York Times,obama,32385.0,4
81404,72143.0,Fact Check Top Lies Obama State Union,President Barack Obama promised final State Un...,Breitbart News,obama,40836.0,4


In [12]:
z=[]
z=list(zip(news_final['IDLink'],news_final['Title'],news_final['Headline'],news_final['Source'],news_final['Topic'],news_final['GooglePlus']))

z1=sorted(z, key=lambda t: t[5])

labels = ['IDLink','Title', 'Headline','Source', 'Topic', 'SharesOfGooglePlus']
df_googlePlus = pd.DataFrame.from_records(z1, columns=labels)

def label_emotion (row):
   if row['SharesOfGooglePlus'] >=0 and row['SharesOfGooglePlus'] <=40  :
      return 1
   if row['SharesOfGooglePlus'] > 40 and row['SharesOfGooglePlus']<=120:
      return 2
   if row['SharesOfGooglePlus'] > 120 and row['SharesOfGooglePlus']<=600:
      return 3
   if row['SharesOfGooglePlus'] >600 and row['SharesOfGooglePlus'] <=605:
      return 4
   if row['SharesOfGooglePlus'] >605 and row['SharesOfGooglePlus'] <=1500:
      return 5

df_googlePlus['category_GooglePlus'] = df_googlePlus.apply (lambda row: label_emotion (row),axis=1)

df_googlePlus


,IDLink,Title,Headline,Source,Topic,SharesOfGooglePlus,category_GooglePlus
0,311.0,Microsoft offering discount original Band UK p...,Just ahead market rollout Microsoft new Band U...,International Business Times via Yahoo UK & Ir...,microsoft,0.0,1
1,252.0,Economy improve though change last months Asso...,quot In coming six months seems growing optimi...,IANS India Private Limited/Yahoo India News vi...,economy,0.0,1
2,227.0,Economy improve though change last months Asso...,New Delhi Nov IANS Majority Indian businesses ...,IANS via Yahoo Maktoob News,economy,0.0,1
3,307.0,Microsoft scales back free online storage plans,NEW YORK Microsoft getting stingy online stora...,The Columbian,microsoft,0.0,1
4,207.0,IDC predicts emergence DX economy,KUCHING The global digital economy enter new s...,The Borneo Post,economy,0.0,1
...,...,...,...,...,...,...,...
81401,2070.0,Intervention PM G working session Inclusive Gr...,The global economic growth remains weak slowdo...,Narendra Modi (press release) (blog),economy,804.0,5
81402,35873.0,Microsoft teen girl AI turns Hitler loving sex...,Developers Microsoft created Tay AI modelled s...,Telegraph.co.uk,microsoft,973.0,5
81403,25397.0,Learning Alphabet,For decades massive market dominated two playe...,The Verge,microsoft,1001.0,5
81404,38485.0,Microsoft adding Linux command line Windows,Microsoft reaching Linux developers way compan...,The Verge,microsoft,1016.0,5


In [13]:
z=[]
z=list(zip(news_final['IDLink'],news_final['Title'],news_final['Headline'],news_final['Source'],news_final['Topic'],news_final['LinkedIn']))

z1=sorted(z, key=lambda t: t[5])

labels = ['IDLink','Title', 'Headline','Source', 'Topic', 'SharesOfLinkedIn']
df_linkedIn = pd.DataFrame.from_records(z1, columns=labels)

def label_emotion (row):
   if row['SharesOfLinkedIn'] >=0 and row['SharesOfLinkedIn'] <=100  :
      return 1
   if row['SharesOfLinkedIn'] > 100 and row['SharesOfLinkedIn']<=2000:
      return 2
   if row['SharesOfLinkedIn'] > 2000 and row['SharesOfLinkedIn']<=6000:
      return 3
   if row['SharesOfLinkedIn'] >6000 and row['SharesOfLinkedIn'] <=6400:
      return 4
df_linkedIn['category_LinkedIn'] = df_linkedIn.apply (lambda row: label_emotion (row),axis=1)

df_linkedIn

,IDLink,Title,Headline,Source,Topic,SharesOfLinkedIn,category_LinkedIn
0,311.0,Microsoft offering discount original Band UK p...,Just ahead market rollout Microsoft new Band U...,International Business Times via Yahoo UK & Ir...,microsoft,0.0,1
1,252.0,Economy improve though change last months Asso...,quot In coming six months seems growing optimi...,IANS India Private Limited/Yahoo India News vi...,economy,0.0,1
2,227.0,Economy improve though change last months Asso...,New Delhi Nov IANS Majority Indian businesses ...,IANS via Yahoo Maktoob News,economy,0.0,1
3,307.0,Microsoft scales back free online storage plans,NEW YORK Microsoft getting stingy online stora...,The Columbian,microsoft,0.0,1
4,207.0,IDC predicts emergence DX economy,KUCHING The global digital economy enter new s...,The Borneo Post,economy,0.0,1
...,...,...,...,...,...,...,...
81401,30049.0,Think tank predicts Manitoba economy one count...,Manitoba economy fare quite well year earlier ...,Winnipeg Free Press,economy,3433.0,3
81402,21084.0,Microsoft Plumbs Ocean Depths Test Underwater ...,REDMOND Wash Taking page Jules Verne researche...,New York Times,microsoft,3652.0,3
81403,35329.0,Canadian economy approaching troubled waters,As federal budget tabled today question Canadi...,Winnipeg Free Press,economy,3716.0,3
81404,41816.0,Hospital wait times costing national economy B,Wait times health treatment costing Canadian e...,Winnipeg Free Press,economy,4328.0,3


In [14]:
from keras.preprocessing.text import one_hot
# integer encode the documents
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in df_facebook['Title']]

In [15]:
res = max(encoded_docs, key = len)
len(res)

25

In [16]:
from keras.utils import pad_sequences
max_length = 5
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [17]:
padded_docs

array([[14, 45, 37, 14, 91],
       [75, 30, 81, 86, 11],
       [75, 30, 81, 86, 11],
       ...,
       [37, 52, 12, 62,  0],
       [74,  3, 62,  5, 71],
       [85, 33, 34, 42, 33]])

In [18]:
x = padded_docs
y = df_facebook['category_facebook']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)


In [19]:
from sklearn.linear_model import LogisticRegression
#Fit logistic regression to the training set
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)

C:\Users\Krish\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [20]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9733195754716981


In [21]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from keras.preprocessing.text import one_hot
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in df_googlePlus['Title']]

In [ ]:
res = max(encoded_docs, key = len)
len(res)

In [ ]:
from keras.utils import pad_sequences
max_length = 25
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
from sklearn.linear_model import LogisticRegression
x = padded_docs
y = df_googlePlus['category_GooglePlus']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
#Fit logistic regression to the training set
classifier = LogisticRegression(random_state=1)
classifier.fit(x_train,y_train)

In [ ]:
#Predicting the test set results and creating confusion matrix
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from keras.preprocessing.text import one_hot#import has not been moved from here for better understandability of the code
# integer encode the documents
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in df_linkedIn['Title']]

In [ ]:
res = max(encoded_docs, key = len)
len(res)

In [ ]:
"""The sequences have different lengths and Keras prefers inputs to be vectorized and all inputs to have the same length. 
We will pad all input sequences to have the length of 4.
Again, we can do this with a built in Keras function, in this case the pad_sequences() function.
pad documents to a max length of 4 words
"""
from keras.utils import pad_sequences
max_length = 25
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
from sklearn.linear_model import LogisticRegression
x = padded_docs
y = df_linkedIn['category_LinkedIn']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
#Fit logistic regression to the training set
classifier = LogisticRegression(random_state=1)
classifier.fit(x_train,y_train)

In [ ]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)